In [1]:
import os
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft


In [3]:
FS = 500                 # Sampling frequency
MAX_LEN = 7500           # ECG length
WINDOW_SIZE = 1000       # 2 seconds
OVERLAP = 0.5
MAX_FREQ = 40            # ECG useful frequency
MAT_DIR = r'D:\ECG_model\Data'
CSV_PATH = r'D:\ECG_model\REFERENCE.csv'
OUTPUT_DIR = r'D:\ECG_model\ecg_images'


In [4]:
def create_ecg_windows(signal, window_size=WINDOW_SIZE, overlap=OVERLAP):
    step = int(window_size * (1 - overlap))
    windows = []
    for start in range(0, len(signal) - window_size + 1, step):
        windows.append(signal[start:start + window_size])
    return windows


In [5]:
def stockwell_transform(signal):
    """
    signal: 1D numpy array (normalized)
    return: 2D Stockwell magnitude (freq x time)
    """
    N = len(signal)
    X = fft(signal)
    t = np.arange(N)

    S = np.zeros((N // 2, N), dtype=np.complex64)

    for k in range(1, N // 2):
        sigma = max(k, 1)
        gaussian = np.exp(-2 * (np.pi ** 2) * ((t - N / 2) ** 2) / (sigma ** 2))
        gaussian = np.roll(gaussian, -N // 2)
        S[k, :] = ifft(X * gaussian)

    return np.abs(S)


In [6]:
def save_stockwell_image(window, save_path):
    # Normalize window (VERY IMPORTANT)
    window = (window - np.mean(window)) / (np.std(window) + 1e-8)

    # Stockwell Transform
    S_mag = stockwell_transform(window)

    # Frequency cropping (0–40 Hz)
    freq_bins = int(MAX_FREQ * WINDOW_SIZE / FS)
    S_mag = S_mag[:freq_bins, :]

    # Log scaling
    S_mag = np.log1p(S_mag)

    # Save image
    plt.figure(figsize=(3, 3))
    plt.imshow(S_mag, aspect="auto", origin="lower", cmap="jet")
    plt.axis("off")
    plt.savefig(save_path, dpi=150, bbox_inches="tight", pad_inches=0)
    plt.close()


In [7]:
labels_df = pd.read_csv(CSV_PATH)
label_dict = dict(zip(labels_df["Recording"], labels_df["First_label"]))


In [8]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

for c in range(1, 10):
    os.makedirs(os.path.join(OUTPUT_DIR, f"class_{c}"), exist_ok=True)


In [9]:
for file in os.listdir(MAT_DIR):
    if not file.endswith(".mat"):
        continue

    record = file.replace(".mat", "")
    if record not in label_dict:
        continue

    label = label_dict[record]

    # Load ECG
    mat = sio.loadmat(os.path.join(MAT_DIR, file))
    signal = mat["ECG"][0][0]["data"][0]   # Lead-1

    # Fix ECG length
    if len(signal) > MAX_LEN:
        signal = signal[:MAX_LEN]
    else:
        signal = np.pad(signal, (0, MAX_LEN - len(signal)))

    # Windowing
    windows = create_ecg_windows(signal)

    # Generate images
    for i, window in enumerate(windows):
        save_path = os.path.join(
            OUTPUT_DIR,
            f"class_{label}",
            f"{record}_win{i}.png"
        )
        save_stockwell_image(window, save_path)

    print(f"Processed {record}")


Processed A0001
Processed A0002
Processed A0003
Processed A0004
Processed A0005
Processed A0006
Processed A0007
Processed A0008
Processed A0009
Processed A0010
Processed A0011
Processed A0012
Processed A0013
Processed A0014
Processed A0015
Processed A0016
Processed A0017
Processed A0018
Processed A0019
Processed A0020
Processed A0021
Processed A0022
Processed A0023
Processed A0024
Processed A0025
Processed A0026
Processed A0027
Processed A0028
Processed A0029
Processed A0030
Processed A0031
Processed A0032
Processed A0033
Processed A0034
Processed A0035
Processed A0036
Processed A0037
Processed A0038
Processed A0039
Processed A0040
Processed A0041
Processed A0042
Processed A0043
Processed A0044
Processed A0045
Processed A0046
Processed A0047
Processed A0048
Processed A0049
Processed A0050
Processed A0051
Processed A0052
Processed A0053
Processed A0054
Processed A0055
Processed A0056
Processed A0057
Processed A0058
Processed A0059
Processed A0061
Processed A0062
Processed A0063
Processe

MemoryError: Unable to allocate 2.44 MiB for an array with shape (80, 1000, 4) and data type float64

Error in callback <function _draw_all_if_interactive at 0x0000020874C72610> (for post_execute), with arguments args (),kwargs {}:


MemoryError: Unable to allocate 2.44 MiB for an array with shape (80, 1000, 4) and data type float64

MemoryError: Unable to allocate 2.44 MiB for an array with shape (80, 1000, 4) and data type float64

<Figure size 300x300 with 1 Axes>

In [2]:
print('I miss you chapri:(..........)')

I miss you chapri:(..........)


In [3]:
import os
import gc
import numpy as np
import pandas as pd
import scipy.io as sio

import matplotlib
matplotlib.use("Agg")   # IMPORTANT: memory safe backend
import matplotlib.pyplot as plt

from scipy.fft import fft, ifft


In [5]:
FS = 500                 # Sampling frequency
MAX_LEN = 7500           # ECG length
WINDOW_SIZE = 1000       # 2 seconds
OVERLAP = 0.5
MAX_FREQ = 40            # ECG useful frequency

MAT_DIR = r'D:\ECG_model\Data'
CSV_PATH = r'D:\ECG_model\REFERENCE.csv'
OUTPUT_DIR = r'D:\ECG_model\ecg_images'


In [6]:
def create_ecg_windows(signal, window_size=WINDOW_SIZE, overlap=OVERLAP):
    step = int(window_size * (1 - overlap))
    windows = []
    for start in range(0, len(signal) - window_size + 1, step):
        windows.append(signal[start:start + window_size])
    return windows


In [7]:
def stockwell_transform(signal):
    N = len(signal)
    X = fft(signal)
    t = np.arange(N)

    S = np.zeros((N // 2, N), dtype=np.complex64)

    for k in range(1, N // 2):
        sigma = max(k, 1)
        gaussian = np.exp(
            -2 * (np.pi ** 2) * ((t - N / 2) ** 2) / (sigma ** 2)
        )
        gaussian = np.roll(gaussian, -N // 2)
        S[k, :] = ifft(X * gaussian)

    return np.abs(S)


In [8]:
def save_stockwell_image(window, save_path):
    # Normalize window
    window = (window - window.mean()) / (window.std() + 1e-8)

    # Stockwell
    S_mag = stockwell_transform(window)

    # Crop to ECG frequencies (0–40 Hz)
    freq_bins = int(MAX_FREQ * len(window) / FS)
    S_mag = S_mag[:freq_bins, :]

    # Log scaling
    S_mag = np.log1p(S_mag)

    # Normalize image to 0–1
    S_mag = (S_mag - S_mag.min()) / (S_mag.max() - S_mag.min() + 1e-8)

    # Save image directly
    plt.imsave(save_path, S_mag, cmap="jet")


In [9]:
labels_df = pd.read_csv(CSV_PATH)
label_dict = dict(zip(labels_df["Recording"], labels_df["First_label"]))


In [10]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

for c in range(1, 10):
    os.makedirs(os.path.join(OUTPUT_DIR, f"class_{c}"), exist_ok=True)


In [11]:
processed = 0
skipped = 0

for file in os.listdir(MAT_DIR):
    if not file.endswith(".mat"):
        continue

    record = file.replace(".mat", "")

    if record not in label_dict:
        continue

    label = label_dict[record]
    class_dir = os.path.join(OUTPUT_DIR, f"class_{label}")

    # 🔴 SKIP if this ECG is already processed
    if any(record in fname for fname in os.listdir(class_dir)):
        print(f"Skipping {record} (already done)")
        skipped += 1
        continue

    # Load ECG
    mat = sio.loadmat(os.path.join(MAT_DIR, file))
    signal = mat["ECG"][0][0]["data"][0]  # Lead-1

    # Fix length
    if len(signal) > MAX_LEN:
        signal = signal[:MAX_LEN]
    else:
        signal = np.pad(signal, (0, MAX_LEN - len(signal)))

    # Windowing
    windows = create_ecg_windows(signal)

    # Generate images
    img_id = 0
    for window in windows:
        save_path = os.path.join(
            class_dir,
            f"{record}_win{img_id}.png"
        )
        save_stockwell_image(window, save_path)
        img_id += 1

    processed += 1
    print(f"Processed {record}")

    # 🔴 MEMORY CLEANUP
    gc.collect()

print("\nDONE")
print("Processed ECGs:", processed)
print("Skipped ECGs:", skipped)


Skipping A0001 (already done)
Skipping A0002 (already done)
Skipping A0003 (already done)
Skipping A0004 (already done)
Skipping A0005 (already done)
Skipping A0006 (already done)
Skipping A0007 (already done)
Skipping A0008 (already done)
Skipping A0009 (already done)
Skipping A0010 (already done)
Skipping A0011 (already done)
Skipping A0012 (already done)
Skipping A0013 (already done)
Skipping A0014 (already done)
Skipping A0015 (already done)
Skipping A0016 (already done)
Skipping A0017 (already done)
Skipping A0018 (already done)
Skipping A0019 (already done)
Skipping A0020 (already done)
Skipping A0021 (already done)
Skipping A0022 (already done)
Skipping A0023 (already done)
Skipping A0024 (already done)
Skipping A0025 (already done)
Skipping A0026 (already done)
Skipping A0027 (already done)
Skipping A0028 (already done)
Skipping A0029 (already done)
Skipping A0030 (already done)
Skipping A0031 (already done)
Skipping A0032 (already done)
Skipping A0033 (already done)
Skipping A

In [12]:
print('simpal')

simpal


In [13]:
import os
import numpy as np
from PIL import Image

IMAGE_DIR = "ecg_images"   # main folder
VAR_THRESHOLD = 5          # VERY SAFE threshold

deleted = 0
kept = 0

for class_folder in os.listdir(IMAGE_DIR):
    class_path = os.path.join(IMAGE_DIR, class_folder)

    if not os.path.isdir(class_path):
        continue

    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)

        try:
            img = Image.open(img_path).convert("L")  # grayscale
            img_array = np.array(img)

            variance = np.var(img_array)

            # DELETE only if image is almost uniform
            if variance < VAR_THRESHOLD:
                os.remove(img_path)
                deleted += 1
            else:
                kept += 1

        except Exception as e:
            print(f"Error reading {img_path}: {e}")

print("\nDONE")
print("Deleted images:", deleted)
print("Kept images:", kept)



DONE
Deleted images: 6125
Kept images: 42650


In [2]:
import os
import shutil
import random

SOURCE_DIR = "ecg_images"
TARGET_DIR = "ecg_images_split"

TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

random.seed(42)

# Create target folders
for split in ["train", "val", "test"]:
    for c in range(1, 10):
        os.makedirs(os.path.join(TARGET_DIR, split, f"class_{c}"), exist_ok=True)

# Split per class
for c in range(1, 10):
    class_dir = os.path.join(SOURCE_DIR, f"class_{c}")
    images = os.listdir(class_dir)
    random.shuffle(images)

    n_total = len(images)
    n_train = int(n_total * TRAIN_RATIO)
    n_val = int(n_total * VAL_RATIO)

    train_imgs = images[:n_train]
    val_imgs = images[n_train:n_train + n_val]
    test_imgs = images[n_train + n_val:]

    for img in train_imgs:
        shutil.copy(
            os.path.join(class_dir, img),
            os.path.join(TARGET_DIR, "train", f"class_{c}", img)
        )

    for img in val_imgs:
        shutil.copy(
            os.path.join(class_dir, img),
            os.path.join(TARGET_DIR, "val", f"class_{c}", img)
        )

    for img in test_imgs:
        shutil.copy(
            os.path.join(class_dir, img),
            os.path.join(TARGET_DIR, "test", f"class_{c}", img)
        )

    print(f"Class {c} done")

print("DATASET SPLIT COMPLETED")


Class 1 done
Class 2 done
Class 3 done
Class 4 done
Class 5 done
Class 6 done
Class 7 done
Class 8 done
Class 9 done
DATASET SPLIT COMPLETED
